# Bitacora para el manejo de secuencias fasta y búsqueda basica de *Blastx*

## Para el siguiente ejercicio es necesario tener el Blast+ instalado en la computadora
https://www.ncbi.nlm.nih.gov/guide/data-software/

In [1]:
cd ~/Desktop/curso/data/exp710/

/Users/migueldelrio/Desktop/curso/data/exp710


In [2]:
ls

710_transcritos.fasta* data/                  results/
710_transcritos.tab    gi*
blastx_borrar.tab*     programs/


In [ ]:
%%bash
export BLASTDB=~/Desktop/bigdata/

date  
time blastx -query 710_transcritos.fasta -db uniprot_sprot \
-out 710_transcritos.tab -evalue 1E-6 -max_target_seqs 1 \
-num_threads 2 -outfmt "6 std stitle" 
date

In [3]:
!head 710_transcritos.tab

1001070759	sp|P46595|UBC4_SCHPO	87.611	113	14	0	7	345	1	113	2.10e-66	199	sp|P46595|UBC4_SCHPO Ubiquitin-conjugating enzyme E2 4 OS=Schizosaccharomyces pombe (strain 972 / ATCC 24843) OX=284812 GN=ubc4 PE=1 SV=1
1001070758	sp|Q9Y4A8|NF2L3_HUMAN	43.243	111	61	1	1031	1357	536	646	2.04e-14	80.1	sp|Q9Y4A8|NF2L3_HUMAN Nuclear factor erythroid 2-related factor 3 OS=Homo sapiens OX=9606 GN=NFE2L3 PE=1 SV=1
1001070756	sp|P22813|HSF_DROME	44.286	70	39	0	621	830	164	233	9.36e-14	75.1	sp|P22813|HSF_DROME Heat shock factor protein OS=Drosophila melanogaster OX=7227 GN=Hsf PE=1 SV=1
1001070755	sp|P02833|ANTP_DROME	90.361	83	4	1	1185	1421	283	365	4.00e-41	157	sp|P02833|ANTP_DROME Homeotic protein antennapedia OS=Drosophila melanogaster OX=7227 GN=Antp PE=1 SV=1
1001070754	sp|G5E8K5|ANK3_MOUSE	25.131	382	203	13	434	1498	352	677	1.78e-12	76.3	sp|G5E8K5|ANK3_MOUSE Ankyrin-3 OS=Mus musculus OX=10090 GN=Ank3 PE=1 SV=1
1001070754	sp|G5E8K5|ANK3_MOUSE	33.077	130	78	3	386	775	469	589	1.43e-10	70.1	sp|G5E8K5|

In [4]:
from pandas import Series, DataFrame
import pandas as pd
from Bio import SeqIO, AlignIO, SeqRecord
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import matplotlib.pyplot as plt 
import os
#from matplotlib_venn import venn3_unweighted

###  Blastx da los resultados sin nombre de columnas, por lo que se asignan a la variable "encabezado". 
### *NOTA:* el blastx a la base de datos swissprot da como segunda columna el identificador de Uniprot y no el del GenBank como en el caso de blastn a la base de datos 16 microbial o nt

In [5]:
# observe el nombre de la segunda columna
encabezado =("qseqid", "sp", "uniprotid", "uniprotid2", "pident", "length", "mismatch", "gapopen","qstart",
             "qend", "sstart","send", "evalue", "bitscore", "stitle")

### cambiando el archivo de tab a csv

In [21]:
f = open ("710_transcritos.tab", 'r')
f_out= open("710_transcritos.csv",'w')
# if you want to add the column name from the blast ftm, to the csv output, eliminate the "#" in the next line
f_out.write('qseqid,sseqid-1,uniprotid,sseqid-3,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,stitle_sp,uniprotid2,stitle,stitle_2'+"\n")
n=0
for line in f.readlines():
    line1 = line.replace("|","\t")
    line1 = line1.replace("\t", ",")
    f_out.write(line1)
    n=n+1

f_out.close()
f.close()
print (str(n)+" sequences processed")

1324 sequences processed


In [22]:
!head 710_transcritos.csv

qseqid,sseqid-1,uniprotid,sseqid-3,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,stitle_sp,uniprotid2,stitle,stitle_2
1001070759,sp,P46595,UBC4_SCHPO,87.611,113,14,0,7,345,1,113,2.10e-66,199,sp,P46595,UBC4_SCHPO Ubiquitin-conjugating enzyme E2 4 OS=Schizosaccharomyces pombe (strain 972 / ATCC 24843) OX=284812 GN=ubc4 PE=1 SV=1
1001070758,sp,Q9Y4A8,NF2L3_HUMAN,43.243,111,61,1,1031,1357,536,646,2.04e-14,80.1,sp,Q9Y4A8,NF2L3_HUMAN Nuclear factor erythroid 2-related factor 3 OS=Homo sapiens OX=9606 GN=NFE2L3 PE=1 SV=1
1001070756,sp,P22813,HSF_DROME,44.286,70,39,0,621,830,164,233,9.36e-14,75.1,sp,P22813,HSF_DROME Heat shock factor protein OS=Drosophila melanogaster OX=7227 GN=Hsf PE=1 SV=1
1001070755,sp,P02833,ANTP_DROME,90.361,83,4,1,1185,1421,283,365,4.00e-41,157,sp,P02833,ANTP_DROME Homeotic protein antennapedia OS=Drosophila melanogaster OX=7227 GN=Antp PE=1 SV=1
1001070754,sp,G5E8K5,ANK3_MOUSE,25.131,382,203,13,434,1498,352,677,1.78e-12,76.3,sp,G5E8K5,ANK3_MOUS

### Se lee el archivo de salida y se asigna a la variable "ftab", con ello se pueden ver los resultados

In [23]:
ftab=pd.read_csv("710_transcritos.csv", header=0,
                  engine='python')
ftab.head()


,qseqid,sseqid-1,uniprotid,sseqid-3,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,stitle_sp,uniprotid2,stitle,stitle_2
0,1001070759,sp,P46595,UBC4_SCHPO,87.611,113,14,0,7,345,1,113,2.100000e-66,199.0,sp,P46595,UBC4_SCHPO Ubiquitin-conjugating enzyme E2 4 O...,None
1,1001070758,sp,Q9Y4A8,NF2L3_HUMAN,43.243,111,61,1,1031,1357,536,646,2.040000e-14,80.1,sp,Q9Y4A8,NF2L3_HUMAN Nuclear factor erythroid 2-related...,None
2,1001070756,sp,P22813,HSF_DROME,44.286,70,39,0,621,830,164,233,9.360000e-14,75.1,sp,P22813,HSF_DROME Heat shock factor protein OS=Drosoph...,None
3,1001070755,sp,P02833,ANTP_DROME,90.361,83,4,1,1185,1421,283,365,4.000000e-41,157.0,sp,P02833,ANTP_DROME Homeotic protein antennapedia OS=Dr...,None
4,1001070754,sp,G5E8K5,ANK3_MOUSE,25.131,382,203,13,434,1498,352,677,1.780000e-12,76.3,sp,G5E8K5,ANK3_MOUSE Ankyrin-3 OS=Mus musculus OX=10090 ...,None


In [37]:
n=0
especie=[]
for row in ftab.index:
    row2=ftab.loc[row]
    try :
        row2["stitle"].find("OS=")
    except:
        linea=row2["stitle_2"][row2["stitle_2"].find("OS=")+3:row2["stitle_2"].find("OX=")]
        print (linea)
        especie.append(linea)

    else:
        linea=row2["stitle"][row2["stitle"].find("OS=")+3:row2["stitle"].find("OX=")]
        print (linea)
        especie.append(linea)
        
    n+=1
    #if n==10:
    #    break
especie[:5]

Schizosaccharomyces pombe (strain 972 / ATCC 24843) 
Homo sapiens 
Drosophila melanogaster 
Drosophila melanogaster 
Mus musculus 
Mus musculus 
Mus musculus 
Mus musculus 
Mus musculus 
Rattus norvegicus 
Mus musculus 
Gallus gallus 
Mus musculus 
Xenopus laevis 
Drosophila melanogaster 
Drosophila melanogaster 
Xenopus tropicalis 
Xenopus tropicalis 
Xenopus tropicalis 
Mus musculus 
Homo sapiens 
Anopheles albimanus 
Drosophila melanogaster 
Drosophila melanogaster 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Drosophila melanogaster 
Drosophila melanogaster 
Drosophila melanogaster 
T1C_ANOGA Glutathione S-transferase 
T1C_ANOGA Glutathione S-transferase 
T1C_ANOGA Glutathione S-transferase 
T1C_ANOGA Glutathione S-transferase 
Drosophila melanogaster 
Drosophila melanogaster 
Drosophila melanogaster 
Drosophila melanogaster 
Drosophila melanogaster 
Drosophila melanogaster 
Drosophila melanogaster 
Drosophila melanogaster 
Drosophila melanogaster 
Drosophi

Homo sapiens 
Xenopus laevis 
Xenopus laevis 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Drosophila melanogaster 
Musca domestica 
Gorilla gorilla gorilla 
Drosophila melanogaster 
Gorilla gorilla gorilla 
Drosophila melanogaster 
Drosophila melanogaster 
Drosophila melanogaster 
Lucilia cuprina 
Xenopus laevis 
Drosophila melanogaster 
Mus musculus 
Spodoptera frugiperda 
Caenorhabditis elegans 
Spodoptera frugiperda 
Mus musculus 
Mus musculus 
Mus musculus 
Drosophila melanogaster 
Canis lupus familiaris 
Blattella germanica 
Rattus norvegicus 
Rattus norvegicus 
Drosophila melanogaster 
Drosophila melanogaster 
Drosophila melanogaster 
Rattus norvegicus 
Rattus norvegicus 
Drosophila melanogaster 
Drosophila melanogaster 
Rattus norvegicus 
Homo sapiens 
Homo sapiens 
Danio rerio 
P25_DROME cGMP-dependent protein kinas
P25_DROME cGMP-dependent protein kinas
P25_DROME cGMP-dependent protein kinas
P25_DROME cGMP-dependent protein kinas
Mus musculus 
Drosophila melanogast

Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 
Homo sapiens 


['Schizosaccharomyces pombe (strain 972 / ATCC 24843) ',
 'Homo sapiens ',
 'Drosophila melanogaster ',
 'Drosophila melanogaster ',
 'Mus musculus ']

In [38]:
ftab["especie"]=especie
ftab.head()

,qseqid,sseqid-1,uniprotid,sseqid-3,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,stitle_sp,uniprotid2,stitle,stitle_2,especie
0,1001070759,sp,P46595,UBC4_SCHPO,87.611,113,14,0,7,345,1,113,2.100000e-66,199.0,sp,P46595,UBC4_SCHPO Ubiquitin-conjugating enzyme E2 4 O...,None,Schizosaccharomyces pombe (strain 972 / ATCC 2...
1,1001070758,sp,Q9Y4A8,NF2L3_HUMAN,43.243,111,61,1,1031,1357,536,646,2.040000e-14,80.1,sp,Q9Y4A8,NF2L3_HUMAN Nuclear factor erythroid 2-related...,None,Homo sapiens
2,1001070756,sp,P22813,HSF_DROME,44.286,70,39,0,621,830,164,233,9.360000e-14,75.1,sp,P22813,HSF_DROME Heat shock factor protein OS=Drosoph...,None,Drosophila melanogaster
3,1001070755,sp,P02833,ANTP_DROME,90.361,83,4,1,1185,1421,283,365,4.000000e-41,157.0,sp,P02833,ANTP_DROME Homeotic protein antennapedia OS=Dr...,None,Drosophila melanogaster
4,1001070754,sp,G5E8K5,ANK3_MOUSE,25.131,382,203,13,434,1498,352,677,1.780000e-12,76.3,sp,G5E8K5,ANK3_MOUSE Ankyrin-3 OS=Mus musculus OX=10090 ...,None,Mus musculus


In [39]:
ftab.tail()

,qseqid,sseqid-1,uniprotid,sseqid-3,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,stitle_sp,uniprotid2,stitle,stitle_2,especie
1319,1001070050,sp,Q8NB50,ZFP62_HUMAN,45.283,53,23,3,2151,2303,647,695,3.830000e-14,35.0,sp,Q8NB50,ZFP62_HUMAN Zinc finger protein 62 homolog OS=...,None,Homo sapiens
1320,1001070050,sp,Q8NB50,ZFP62_HUMAN,25.188,532,314,20,584,2032,198,694,2.510000e-13,70.1,sp,Q8NB50,ZFP62_HUMAN Zinc finger protein 62 homolog OS=...,None,Homo sapiens
1321,1001070050,sp,Q8NB50,ZFP62_HUMAN,42.105,57,26,4,2151,2315,731,782,2.510000e-13,28.5,sp,Q8NB50,ZFP62_HUMAN Zinc finger protein 62 homolog OS=...,None,Homo sapiens
1322,1001070050,sp,Q8NB50,ZFP62_HUMAN,27.248,367,182,13,983,2032,563,861,1.370000e-11,72.4,sp,Q8NB50,ZFP62_HUMAN Zinc finger protein 62 homolog OS=...,None,Homo sapiens
1323,1001070050,sp,Q8NB50,ZFP62_HUMAN,26.667,360,222,12,983,2032,479,806,2.430000e-11,71.2,sp,Q8NB50,ZFP62_HUMAN Zinc finger protein 62 homolog OS=...,None,Homo sapiens


In [40]:
ftab1= ftab.groupby("especie")["qseqid"].count()
ftab1 = DataFrame(ftab1)
ftab1

,qseqid
especie,
100_BOVIN Protein PET100 homolo,4
2A5_DROME Probable cytochrome P450 12a,1
315_DROME Cytochrome P450 315a,1
61_RAT 6-phosphofructo-2-kinase/fructose-,1
Anopheles albimanus,1
Arabidopsis thaliana,3
Artemia franciscana,4
Ascaris suum,1
BCH_CHICK 3-hydroxyisobutyryl-CoA hydrolas,1


In [42]:
ftab1 = ftab.drop_duplicates(subset = "qseqid", inplace = False)
ftab1.describe().round(3)

,qseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
count,4.390000e+02,439.000,439.000,439.000,439.000,439.000,439.000,439.000,439.000,439.0,439.000
mean,1.001070e+09,52.885,255.916,113.237,3.376,853.961,849.854,213.018,460.957,0.0,245.444
std,1.979230e+02,18.030,211.558,106.592,4.012,1052.830,908.053,405.448,432.854,0.0,203.528
min,1.001070e+09,22.751,37.000,4.000,0.000,1.000,1.000,1.000,49.000,0.0,34.700
25%,1.001070e+09,36.607,109.000,42.000,0.000,137.000,242.500,12.000,200.000,0.0,89.000
50%,1.001070e+09,49.315,185.000,75.000,2.000,434.000,567.000,58.000,341.000,0.0,177.000
75%,1.001071e+09,65.678,336.000,136.000,4.000,1139.000,1179.000,230.000,547.000,0.0,325.000
max,1.001071e+09,95.238,1130.000,634.000,19.000,6725.000,5499.000,2302.000,2688.000,0.0,1066.000


In [43]:
ftab2= ftab1.groupby("especie")["qseqid"].count()
ftab2 = DataFrame(ftab2)
ftab2

,qseqid
especie,
100_BOVIN Protein PET100 homolo,4
2A5_DROME Probable cytochrome P450 12a,1
315_DROME Cytochrome P450 315a,1
61_RAT 6-phosphofructo-2-kinase/fructose-,1
Anopheles albimanus,1
Arabidopsis thaliana,3
Artemia franciscana,4
Ascaris suum,1
BCH_CHICK 3-hydroxyisobutyryl-CoA hydrolas,1


In [47]:
encabezadospid= ("uniprotid", "go")

In [48]:
fspid = pd.read_table('data/slim.txt', header= None, names= encabezadospid, delimiter="\t", engine="python")
fspid.head(2)

,uniprotid,go
0,A0A023PZB3,GO:0005739
1,A0A023PZB3,GO:0005739


In [51]:
ftab1.head()

,qseqid,sseqid-1,uniprotid,sseqid-3,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,stitle_sp,uniprotid2,stitle,stitle_2,especie
0,1001070759,sp,P46595,UBC4_SCHPO,87.611,113,14,0,7,345,1,113,2.100000e-66,199.0,sp,P46595,UBC4_SCHPO Ubiquitin-conjugating enzyme E2 4 O...,None,Schizosaccharomyces pombe (strain 972 / ATCC 2...
1,1001070758,sp,Q9Y4A8,NF2L3_HUMAN,43.243,111,61,1,1031,1357,536,646,2.040000e-14,80.1,sp,Q9Y4A8,NF2L3_HUMAN Nuclear factor erythroid 2-related...,None,Homo sapiens
2,1001070756,sp,P22813,HSF_DROME,44.286,70,39,0,621,830,164,233,9.360000e-14,75.1,sp,P22813,HSF_DROME Heat shock factor protein OS=Drosoph...,None,Drosophila melanogaster
3,1001070755,sp,P02833,ANTP_DROME,90.361,83,4,1,1185,1421,283,365,4.000000e-41,157.0,sp,P02833,ANTP_DROME Homeotic protein antennapedia OS=Dr...,None,Drosophila melanogaster
4,1001070754,sp,G5E8K5,ANK3_MOUSE,25.131,382,203,13,434,1498,352,677,1.780000e-12,76.3,sp,G5E8K5,ANK3_MOUSE Ankyrin-3 OS=Mus musculus OX=10090 ...,None,Mus musculus


In [52]:
f2=pd.merge(ftab1,fspid, on ="uniprotid", how='inner')
f2.head(2)

,qseqid,sseqid-1,uniprotid,sseqid-3,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,stitle_sp,uniprotid2,stitle,stitle_2,especie,go


In [ ]:
fspid = ''
fspid

In [ ]:
!date

In [ ]:
!date

In [ ]:
!date
fgo = pd.read_csv('~/Desktop/bigdata/go_to_goslim.csv', engine="python")
fgo.head(2)

In [ ]:
!date

In [ ]:
f3=pd.merge(f2,fgo, on ="GO_id" , how='inner')
f3.head()

In [ ]:
!date

In [ ]:
f4=f3.drop_duplicates(subset = ('qseqid', "aspect"), inplace = False)
f4.describe()[['length','evalue']]

In [ ]:
f4.to_csv("710_transcritos_goslim.csv", index =  None)

In [ ]:
ftabpivot = f4.pivot_table(values="uniprotid" , index=["qseqid"], aggfunc=len, columns="aspect")
ftabpivot.describe()

# Proceso para generar el diagama de Venn con la información de 
## Componentes celulares, funciones biologicas y procesos biologicos, C, F y P, respectivamente

In [ ]:
lineaC =[] # data from C
lineaF =[] # data from F
lineaP =[] # data from P
linea = ""
n=1
for row in ftabpivot.index:
    row2=ftabpivot.loc[row]
    if str(row2["C"])=="nan" and str(row2["F"])=="nan" and str(row2["P"])=="nan" :
        continue    
    else:        
        if str(row2["C"]) !="nan":
            linea = row
        else:
            linea = ""
        lineaC.append(linea)
        if str(row2["F"]) !="nan":
            linea = row
        else:
            linea = ""
        lineaF.append(linea)

        if str(row2["P"]) !="nan":
            linea = row
        else:
            linea = ""
        lineaP.append(linea)

        n+=1
        #if n==1000:
        #    break

len(lineaC), len(lineaF), len(lineaP)

In [ ]:
lineaC = set(lineaC)
lineaF = set(lineaF)
lineaP = set(lineaP)
venn3_unweighted([lineaC, lineaF, lineaP], ('C', 'F', 'P'))
plt.savefig("710_transcritos_venn3_1.png", dpi=400, bbox_inches='tight')
plt.savefig("710_transcritos_venn3_1.pdf", dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
fgo=f4.groupby('GOSlim_bin')["qseqid"].count()
#fgo

fgo.sort_values(inplace = True, ascending=False)
#fgo

linea10=fgo[0:10]
linea11=fgo[10:]
#linea10

#linea11
otro=sum (linea11)
#otro
otros = pd.DataFrame({0:otro}, index=["Other"])
#otros
linea10=linea10.append(otros)
#linea10
linea10.plot(kind='barh', color=list('ybg'))
plt.axis([-1, 500, -1, 11], label=None)
plt.xlabel("Count")
plt.ylabel("GOSlim bin")
plt.legend().set_visible(False)
#plt.savefig("710transcritos_blastx_GObar.png", dpi=400, bbox_inches='tight')


plt.show()